In [1]:
from datasets import load_dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Subset
from transformers import EarlyStoppingCallback
from sklearn.model_selection import StratifiedKFold 
import numpy as np
from datasets import Dataset, DatasetDict, ClassLabel

/home/elson/factcheck/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('csv',data_files='dataset_semanticattribution_nerfeatures_split.csv',delimiter=',',column_names=["claim","premise","label","category","count_bf","count_ca","count_dis","count_food","count_lipid","count_treat","pres_bf","pres_ca","pres_dis","pres_food","pres_lipid","pres_treat","counte_bf","counte_ca","counte_dis","counte_food","counte_lipid","counte_treat","prese_bf","prese_ca","prese_dis","prese_food","prese_lipid","prese_treat","url", "entities","entity_map","gold_exp","gem_exp","gem_label","entity_map_ev","entity_ev","split"],skiprows=1)

Using custom data configuration default-a34873d1954befad
Reusing dataset csv (/home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|██████████| 1/1 [00:00<00:00, 203.23it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['claim', 'premise', 'label', 'category', 'count_bf', 'count_ca', 'count_dis', 'count_food', 'count_lipid', 'count_treat', 'pres_bf', 'pres_ca', 'pres_dis', 'pres_food', 'pres_lipid', 'pres_treat', 'counte_bf', 'counte_ca', 'counte_dis', 'counte_food', 'counte_lipid', 'counte_treat', 'prese_bf', 'prese_ca', 'prese_dis', 'prese_food', 'prese_lipid', 'prese_treat', 'url', 'entities', 'entity_map', 'gold_exp', 'gem_exp', 'gem_label', 'entity_map_ev', 'entity_ev', 'split'],
        num_rows: 2322
    })
})

In [4]:
dataset['train']['split']

['train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'validation',
 'train',
 'validation',
 'validation',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'test',
 'validation',
 'test',
 'validation',
 'train',
 'validation',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'test',
 'validation',
 'train',
 'validation',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'validation',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'train',
 'valida

In [5]:
train_dataset = dataset['train'].filter(lambda example: example['split'] == 'train')
validation_dataset = dataset['train'].filter(lambda example: example['split'] == 'validation')
test_dataset = dataset['train'].filter(lambda example: example['split'] == 'test')
dataset = DatasetDict({
    'train': train_dataset,
    'val': validation_dataset,
    'test': test_dataset
})

Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-bef253756eb081e8.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-d6584f8c48cc3aa1.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-76c59325f3af690a.arrow


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['claim', 'premise', 'label', 'category', 'count_bf', 'count_ca', 'count_dis', 'count_food', 'count_lipid', 'count_treat', 'pres_bf', 'pres_ca', 'pres_dis', 'pres_food', 'pres_lipid', 'pres_treat', 'counte_bf', 'counte_ca', 'counte_dis', 'counte_food', 'counte_lipid', 'counte_treat', 'prese_bf', 'prese_ca', 'prese_dis', 'prese_food', 'prese_lipid', 'prese_treat', 'url', 'entities', 'entity_map', 'gold_exp', 'gem_exp', 'gem_label', 'entity_map_ev', 'entity_ev', 'split'],
        num_rows: 1623
    })
    val: Dataset({
        features: ['claim', 'premise', 'label', 'category', 'count_bf', 'count_ca', 'count_dis', 'count_food', 'count_lipid', 'count_treat', 'pres_bf', 'pres_ca', 'pres_dis', 'pres_food', 'pres_lipid', 'pres_treat', 'counte_bf', 'counte_ca', 'counte_dis', 'counte_food', 'counte_lipid', 'counte_treat', 'prese_bf', 'prese_ca', 'prese_dis', 'prese_food', 'prese_lipid', 'prese_treat', 'url', 'entities', 'entity_map', 'gold_

In [7]:
columns_to_keep = ["claim", "premise", "label","category"]
all_columns = dataset["train"].column_names

columns_to_drop = [col for col in all_columns if col not in columns_to_keep]
for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_drop)

In [8]:
from datasets import load_dataset, DatasetDict

label2id = {
    "contradiction": 1,
    "entailment": 0,
    "neutral": 1
}

id2label = {v: k for k, v in label2id.items()}

label_mapping = {
    'SUPPORTED': 'entailment',
    'REFUTED': 'contradiction',
    'NOT ENOUGH INFORMATION': 'neutral'
}

def map_and_encode_labels(example):
    # Map original dataset labels to new labels ('entailment', 'contradiction', 'neutral')
    mapped_label = label_mapping[example['label']]
    # Encode mapped labels using label2id
    example['label'] = label2id[mapped_label]
    return example

for split in dataset.keys():
    dataset[split] = dataset[split].map(map_and_encode_labels)

# Show the label encoding mapping
print("Label Encoding Mapping:", label2id)

Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-e86a37205ff67ef4.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-a4d308ee8d45450d.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-a34873d1954befad/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-d183aa926e132b4f.arrow


Label Encoding Mapping: {'contradiction': 1, 'entailment': 0, 'neutral': 1}


In [9]:
labels = np.array(dataset['train']['label'])

In [10]:
from transformers import AutoTokenizer
import torch.utils.data

class MediClaimDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer_name='MoritzLaurer/DeBERTa-v3-base-mnli-fever-docnli-ling-2c'):
        self.dataset = dataset
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        idx = int(idx)  # Ensure idx is an integer
        item = self.dataset[idx]  # Access the dataset item at idx
        
        # Extracting claim and evidence texts

        claim = item['claim']
        category = item['category']
        premise = item['premise'].replace('\n', '')
        item['claim']=claim + "[SEP]" + category
        item['premise']=premise
        
        # Tokenize the texts
        inputs = self.tokenizer(
            premise, claim,
            return_tensors="pt",  # Ensure PyTorch tensors are returned
            padding='max_length',  # Apply padding to the maximum length
            truncation='longest_first',  # Truncate to the maximum length if necessary
            max_length=512,  # Specify the maximum length
            add_special_tokens=True  # Add special tokens like [CLS], [SEP]
        )
        
        item['input_ids'] = inputs['input_ids'].squeeze()  # Remove batch dimension
        item['attention_mask']= inputs['attention_mask'].squeeze() # Remove batch dimension
        
        if 'label' in item:
            item['labels'] = torch.tensor(item['label'], dtype=torch.long)
        
        return item



In [11]:
import torch
print(torch.cuda.device_count())
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

1
Available GPUs:
GPU 0: Tesla V100-SXM2-32GB


In [12]:
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-docnli-ling-2c"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                 num_labels=2, ignore_mismatched_sizes=True)
device = "cuda:0"
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average="weighted")  # Specify average method
    recall = recall_score(labels, preds, average="weighted")  # Specify average method

    bal_accuracy = balanced_accuracy_score(labels,preds)

    return {"accuracy": acc, "balanced_accuracy":bal_accuracy, "precision": prec, "recall": recall, "f1": f1}

In [14]:
dataset['train']

Dataset({
    features: ['claim', 'premise', 'label', 'category'],
    num_rows: 1623
})

In [15]:
import gc

torch.cuda.set_device(0)

# Clearing the cache
torch.cuda.empty_cache()
gc.collect()
# Checking GPU memory, making sure to reset peak memory stats
torch.cuda.reset_peak_memory_stats()

In [16]:
current_device = torch.cuda.current_device()
print(f"Current CUDA device: GPU {current_device}")

Current CUDA device: GPU 0


In [17]:
train_data = dataset['train']
eval_data = dataset['val']
model = model.to('cuda:0')

In [18]:
tdata = MediClaimDataset(train_data)
vdata = MediClaimDataset(eval_data)
test_data = MediClaimDataset(dataset['test'])

In [19]:
tdata.__getitem__(0)

{'claim': 'Myrrh essential oil is sometimes used in skincare products to help improve the appearance of the skin.[SEP]General Health',
 'premise': '; Unković, N.; Dimkić, I.; Janaćković, P.; Gavrilović, M.; Stanojević, O.; Vukojević, J. Frankincense and myrrh essential oils and burn incense fume against micro-inhabitants of sacral ambients.Shameem, I. Phytochemical & therapeutic potentials of Murr makki (.Oxidative stress and immunotoxic effects of lead and their amelioration with myrrh (Commiphora molmol) emulsion.Essential Oils: Magical Ingredients for Skin Care.Chakravarty, N.; Kellogg, C.; Alvarez, J.; Equils, O.; Morgan, M. UV Protection by Natural Products: C. myrrha Oil Versus Sunscreen.Hamidpour, R.; Hamidpour, S.; Hamidpour, M.; Shahlari, M. Frankincense (乳香 Rǔ Xiāng;.species): From the selection of traditional applications to the novel phytotherapy for the prevention and treatment of serious diseases.Chemistry and immunomodulatory activity of frankincense oil.Compositions con

In [20]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments,DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


training_args = TrainingArguments(
    output_dir=f'/home/elson/5.2.2_deberta_docnli/',
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    dataloader_pin_memory=True,
    dataloader_num_workers=4,
    fp16=True,
    warmup_ratio=0.06,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tdata,
    eval_dataset=vdata,
    #tokenizer=tokenizer,
    #data_collator = data_collator,
    compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)]
)

# Training and Evaluation
trainer.train()
eval_result = trainer.evaluate(vdata)

# Save the best model and tokenizer
model.save_pretrained(f'/home/elson/5.2.2_deberta_docnli/best_model')
tokenizer.save_pretrained(f'/home/elson/5.2.2_deberta_docnli/best_model')


Using amp half precision backend
/home/elson/factcheck/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1623
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1530


Epoch,Training Loss,Validation Loss,Accuracy,Balanced Accuracy,Precision,Recall,F1
1,0.474800,0.752921,0.679570,0.704731,0.746034,0.679570,0.691625
2,0.189400,1.491669,0.612903,0.660274,0.718467,0.612903,0.625571
3,0.079500,1.471207,0.711828,0.705121,0.741251,0.711828,0.720322
4,0.040000,1.714582,0.750538,0.708268,0.750538,0.750538,0.750538
5,0.032900,2.404559,0.698925,0.680458,0.721160,0.698925,0.706402
6,0.000100,2.460643,0.705376,0.688960,0.728117,0.705376,0.712845
7,0.000200,2.632856,0.666667,0.683898,0.727116,0.666667,0.679113
8,0.000100,2.234702,0.718280,0.700221,0.737642,0.718280,0.724821
9,0.000100,2.434311,0.711828,0.695548,0.733595,0.711828,0.718985
10,0.000100,2.516026,0.713978,0.697105,0.734936,0.713978,0.720931


***** Running Evaluation *****
  Num examples = 465
  Batch size = 16
Saving model checkpoint to /home/elson/5.2.2_deberta_docnli/checkpoint-102
Configuration saved in /home/elson/5.2.2_deberta_docnli/checkpoint-102/config.json
Model weights saved in /home/elson/5.2.2_deberta_docnli/checkpoint-102/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 465
  Batch size = 16
Saving model checkpoint to /home/elson/5.2.2_deberta_docnli/checkpoint-204
Configuration saved in /home/elson/5.2.2_deberta_docnli/checkpoint-204/config.json
Model weights saved in /home/elson/5.2.2_deberta_docnli/checkpoint-204/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 465
  Batch size = 16
Saving model checkpoint to /home/elson/5.2.2_deberta_docnli/checkpoint-306
Configuration saved in /home/elson/5.2.2_deberta_docnli/checkpoint-306/config.json
Model weights saved in /home/elson/5.2.2_deberta_docnli/checkpoint-306/pytorch_model.bin
Deleting older checkpoint [/home/elson/5.2.2_debe

Configuration saved in /home/elson/5.2.2_deberta_docnli/best_model/config.json
Model weights saved in /home/elson/5.2.2_deberta_docnli/best_model/pytorch_model.bin
tokenizer config file saved in /home/elson/5.2.2_deberta_docnli/best_model/tokenizer_config.json
Special tokens file saved in /home/elson/5.2.2_deberta_docnli/best_model/special_tokens_map.json
added tokens file saved in /home/elson/5.2.2_deberta_docnli/best_model/added_tokens.json


('/home/elson/5.2.2_deberta_docnli/best_model/tokenizer_config.json',
 '/home/elson/5.2.2_deberta_docnli/best_model/special_tokens_map.json',
 '/home/elson/5.2.2_deberta_docnli/best_model/spm.model',
 '/home/elson/5.2.2_deberta_docnli/best_model/added_tokens.json')

In [21]:
model_path = "/home/elson/5.2.2_deberta_docnli/best_model/"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to('cuda:0')

# Evaluate on the test set
test_results = trainer.predict(test_data)

loading configuration file /home/elson/5.2.2_deberta_docnli/best_model/config.json
Model config DebertaV2Config {
  "_name_or_path": "/home/elson/5.2.2_deberta_docnli/best_model/",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "not_entailment"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "entailment": 0,
    "not_entailment": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention

In [22]:
print(test_results)

PredictionOutput(predictions=array([[-0.862 ,  0.809 ],
       [ 5.    , -4.69  ],
       [ 4.86  , -4.566 ],
       [-4.06  ,  4.266 ],
       [-1.611 ,  1.659 ],
       [ 5.203 , -4.875 ],
       [ 4.426 , -4.188 ],
       [ 4.684 , -4.41  ],
       [ 5.16  , -4.836 ],
       [ 3.963 , -3.766 ],
       [ 5.023 , -4.71  ],
       [ 5.133 , -4.812 ],
       [ 4.06  , -3.852 ],
       [ 5.18  , -4.85  ],
       [ 4.73  , -4.453 ],
       [-3.678 ,  3.88  ],
       [ 3.674 , -3.496 ],
       [ 4.75  , -4.47  ],
       [ 4.934 , -4.637 ],
       [ 4.82  , -4.54  ],
       [-0.752 ,  0.801 ],
       [ 2.643 , -2.518 ],
       [ 5.164 , -4.844 ],
       [-4.29  ,  4.504 ],
       [ 3.965 , -3.766 ],
       [-4.875 ,  5.04  ],
       [ 5.14  , -4.812 ],
       [ 4.855 , -4.562 ],
       [ 4.39  , -4.15  ],
       [ 5.465 , -5.1   ],
       [-1.217 ,  1.232 ],
       [ 4.33  , -4.094 ],
       [ 4.664 , -4.39  ],
       [ 4.195 , -3.967 ],
       [-3.479 ,  3.652 ],
       [ 4.863 , -4.57  ],